# RecSys проект для прохождения отбора на стажировку в Т-Банк

## Описание проекта

В рамках данного проекта разрабатывается двухуровневая рекомендательная система, ориентированная на сценарий рекомендательной ленты в цифровом продукте. Целью проекта является построение масштабируемого решения, способного эффективно работать с большим числом пользователей и объектов, а также учитывать как поведенческие данные, так и контентные признаки.

В качестве первого уровня используется модель candidate generation на основе матричной факторизации (LightFM), предназначенная для быстрого отбора релевантных кандидатов из полного множества объектов. На втором уровне реализуется модель ранжирования на основе градиентного бустинга (LightGBM), которая переупорядочивает отобранные кандидаты с учётом дополнительных признаков пользователей, объектов и их взаимодействий.

Проект реализован на реальных данных Amazon Reviews 2023 и включает полный цикл работы с данными: загрузку и очистку, фильтрацию холодных пользователей и объектов, временное разбиение данных, обучение моделей и оценку качества рекомендаций. Архитектура решения и используемые подходы приближены к практикам, применяемым в рекомендательных системах банковских и финтех-продуктов.

## Цель проекта

Целью проекта является построение и исследование масштабируемой рекомендательной системы, способной:

	•	эффективно работать с большим количеством пользователей и объектов;
	•	учитывать как поведенческие данные, так и контентные признаки;
	•	обеспечивать улучшение качества рекомендаций по сравнению с простыми бейзлайнами.

В рамках проекта предполагается:

	•	реализовать двухуровневую архитектуру рекомендаций;
	•	оценить вклад каждого уровня в итоговое качество;
	•	проанализировать полученные результаты и выявить направления для дальнейшего улучшения модели.

## Работа с данными

### Выбор источника данных

Для обучения и оценки рекомендательной системы используется датасет Amazon Reviews 2023, содержащий:

	•	пользовательские отзывы и рейтинги;
	•	временные метки взаимодействий;
	•	метаданные товаров (категория, цена, описание, бренд и др.).

Для снижения размерности задачи и приближения к реальному продуктовому сценарию выбирается одна доменная категория товаров.

### Скачивание данных

В Google Colab данные загружаются напрямую из официального репозитория Amazon Reviews 2023. Для выбранной категории Electronics скачиваются два файла:

	•	Отзывы пользователей (review)
	•	Метаданные товаров (meta)

Оба файла имеют формат JSONL (JSON Lines), что позволяет обрабатывать данные построчно, не загружая весь массив в память сразу, что особенно важно для больших датасетов.

In [ ]:
import os
import urllib.request
import gzip
from google.colab import drive

if not os.path.exists("/content/drive/MyDrive"):
    drive.mount("/content/drive")

data_dir = "/content/drive/MyDrive/RecSys project T-Bank"

review_url = "https://mcauleylab.ucsd.edu/public_datasets/data/amazon_2023/raw/review_categories/Electronics.jsonl.gz"
meta_url = "https://mcauleylab.ucsd.edu/public_datasets/data/amazon_2023/raw/meta_categories/meta_Electronics.jsonl.gz"

review_gz_path = os.path.join(data_dir, "Electronics.jsonl.gz")
meta_gz_path = os.path.join(data_dir, "meta_Electronics.jsonl.gz")

review_path = os.path.join(data_dir, "Electronics_sample.jsonl")
meta_path = os.path.join(data_dir, "meta_Electronics_sample.jsonl")

MAX_ROWS = 250_000  # 250 тысяч строк

# Функция прогресса загрузки
def download_with_progress(url, output_path):
    def show_progress(block_num, block_size, total_size):
        downloaded = block_num * block_size
        percent = min(100, downloaded / total_size * 100)
        print(f"\r{os.path.basename(output_path)}: {percent:.2f}% ({downloaded/1e6:.1f}/{total_size/1e6:.1f} MB)", end="")
    urllib.request.urlretrieve(url, output_path, reporthook=show_progress)
    print("\nЗагрузка завершена!")

# Скачиваем файлы с прогрессом
download_with_progress(review_url, review_gz_path)
download_with_progress(meta_url, meta_gz_path)



Electronics.jsonl.gz: 100.00% (6474.4/6474.4 MB)
Загрузка завершена!
meta_Electronics.jsonl.gz: 100.00% (1312.9/1312.9 MB)
Загрузка завершена!


In [ ]:
# Распаковка с ограничением строк
def gunzip_with_limit(input_path, output_path, max_rows):
    with gzip.open(input_path, "rt", encoding="utf-8") as f_in, \
         open(output_path, "w", encoding="utf-8") as f_out:
        for i, line in enumerate(f_in):
            if i >= max_rows:
                break
            f_out.write(line)

print(f"Распаковываем reviews ({MAX_ROWS} строк)...")
gunzip_with_limit(review_gz_path, review_path, MAX_ROWS)

print(f"Распаковываем meta ({MAX_ROWS} строк)...")
gunzip_with_limit(meta_gz_path, meta_path, MAX_ROWS)

# Проверка
print("\nГотово. Файлы в директории:")
for f in os.listdir(data_dir):
    print(f)

Распаковываем reviews (250000 строк)...
Распаковываем meta (250000 строк)...

Готово. Файлы в директории:
RecSys project T-Bank.ipynb
Electronics.jsonl.gz
meta_Electronics.jsonl.gz
Electronics_sample.jsonl
meta_Electronics_sample.jsonl


In [ ]:
# Загрузка первых строк reviews (Electronics)

import pandas as pd

reviews_df = pd.read_json(
    review_gz_path,
    lines=True,
    compression="gzip",
    nrows=5
)

reviews_df

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,3,Smells like gasoline! Going back!,First & most offensive: they reek of gasoline ...,[{'small_image_url': 'https://m.media-amazon.c...,B083NRGZMM,B083NRGZMM,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-07-18 22:58:37.948,0,True
1,1,Didn’t work at all lenses loose/broken.,These didn’t work. Idk if they were damaged in...,[],B07N69T6TM,B07N69T6TM,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2020-06-20 18:42:29.731,0,True
2,5,Excellent!,I love these. They even come with a carry case...,[],B01G8JO5F2,B01G8JO5F2,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2018-04-07 09:23:37.534,0,True
3,5,Great laptop backpack!,I was searching for a sturdy backpack for scho...,[],B001OC5JKY,B001OC5JKY,AGGZ357AO26RQZVRLGU4D4N52DZQ,2010-11-20 18:41:35.000,18,True
4,5,Best Headphones in the Fifties price range!,I've bought these headphones three times becau...,[],B013J7WUGC,B07CJYMRWM,AG2L7H23R5LLKDKLBEF2Q3L2MVDA,2023-02-17 02:39:41.238,0,True


In [ ]:
# Загрузка первых строк meta (товары)

import gzip
import json

rows = []

with gzip.open(meta_gz_path, "rt", encoding="utf-8") as f:
    for i, line in enumerate(f):
        try:
            rows.append(json.loads(line))
        except json.JSONDecodeError:
            continue
        if len(rows) == 5:
            break

meta_df = pd.DataFrame(rows)
meta_df

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together
0,All Electronics,FS-1051 FATSHARK TELEPORTER V3 HEADSET,3.5,6,[],[Teleporter V3 The “Teleporter V3” kit sets a ...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Fat Shark,"[Electronics, Television & Video, Video Glasses]","{'Date First Available': 'August 2, 2014', 'Ma...",B00MCW7G9M,None
1,All Electronics,Ce-H22B12-S1 4Kx2K Hdmi 4Port,5.0,1,"[UPC: 662774021904, Weight: 0.600 lbs]",[HDMI In - HDMI Out],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],SIIG,"[Electronics, Television & Video, Accessories,...",{'Product Dimensions': '0.83 x 4.17 x 2.05 inc...,B00YT6XQSE,None
2,Computers,Digi-Tatoo Decal Skin Compatible With MacBook ...,4.5,246,[WARNING: Please IDENTIFY MODEL NUMBER on the ...,[],19.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'AL 2Sides Video', 'url': 'https://...",Digi-Tatoo,"[Electronics, Computers & Accessories, Laptop ...","{'Brand': 'Digi-Tatoo', 'Color': 'Fresh Marble...",B07SM135LS,None
3,AMAZON FASHION,NotoCity Compatible with Vivoactive 4 band 22m...,4.5,233,[☛NotoCity 22mm band is designed for Vivoactiv...,[],9.99,[{'thumb': 'https://m.media-amazon.com/images/...,[],NotoCity,"[Electronics, Wearable Technology, Clips, Arm ...","{'Date First Available': 'May 29, 2020', 'Manu...",B089CNGZCW,None
4,Cell Phones & Accessories,Motorola Droid X Essentials Combo Pack,3.8,64,"[New Droid X Essentials Combo Pack, Exclusive ...",[all Genuine High Quality Motorola Made Access...,14.99,[{'thumb': 'https://m.media-amazon.com/images/...,[],Verizon,"[Electronics, Computers & Accessories, Compute...",{'Product Dimensions': '11.6 x 6.9 x 3.1 inche...,B004E2Z88O,None


In [ ]:
# Посмотреть колонки (нужно важно для фичей)

print("Review columns:")
print(reviews_df.columns)

print("\nMeta columns:")
print(meta_df.columns)

Review columns:
Index(['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase'],
      dtype='object')

Meta columns:
Index(['main_category', 'title', 'average_rating', 'rating_number', 'features',
       'description', 'price', 'images', 'videos', 'store', 'categories',
       'details', 'parent_asin', 'bought_together'],
      dtype='object')


In [ ]:
# Быстрый осмотр типов

reviews_df.info()
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   rating             5 non-null      int64         
 1   title              5 non-null      object        
 2   text               5 non-null      object        
 3   images             5 non-null      object        
 4   asin               5 non-null      object        
 5   parent_asin        5 non-null      object        
 6   user_id            5 non-null      object        
 7   timestamp          5 non-null      datetime64[ns]
 8   helpful_vote       5 non-null      int64         
 9   verified_purchase  5 non-null      bool          
dtypes: bool(1), datetime64[ns](1), int64(2), object(6)
memory usage: 497.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ----

In [ ]:
# Понять потенциал фичей

reviews_df.sample(3)
meta_df.sample(3)

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together
4,Cell Phones & Accessories,Motorola Droid X Essentials Combo Pack,3.8,64,"[New Droid X Essentials Combo Pack, Exclusive ...",[all Genuine High Quality Motorola Made Access...,14.99,[{'thumb': 'https://m.media-amazon.com/images/...,[],Verizon,"[Electronics, Computers & Accessories, Compute...",{'Product Dimensions': '11.6 x 6.9 x 3.1 inche...,B004E2Z88O,None
1,All Electronics,Ce-H22B12-S1 4Kx2K Hdmi 4Port,5.0,1,"[UPC: 662774021904, Weight: 0.600 lbs]",[HDMI In - HDMI Out],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],SIIG,"[Electronics, Television & Video, Accessories,...",{'Product Dimensions': '0.83 x 4.17 x 2.05 inc...,B00YT6XQSE,None
3,AMAZON FASHION,NotoCity Compatible with Vivoactive 4 band 22m...,4.5,233,[☛NotoCity 22mm band is designed for Vivoactiv...,[],9.99,[{'thumb': 'https://m.media-amazon.com/images/...,[],NotoCity,"[Electronics, Wearable Technology, Clips, Arm ...","{'Date First Available': 'May 29, 2020', 'Manu...",B089CNGZCW,None


### Первичная обработка данных
  
На этапе первичной обработки выполняются:

	•	чтение и парсинг JSONL-файлов с отзывами и метаданными товаров;
	•	извлечение ключевых полей из отзывов (user_id, parent_asin, rating, timestamp, verified_purchase);
	•	объединение взаимодействий пользователей с метаданными товаров по parent_asin;
	•	фильтрация по рейтингу (оставляем только рейтинги ≥ 3);
	•	удаление пользователей и товаров с малым количеством взаимодействий;
	•	приведение колонок с числовыми значениями (price) к корректному типу;
	•	сортировка взаимодействий по времени;
	•	сохранение очищенных таблиц в формате Parquet для дальнейшей работы.

In [ ]:
import gzip
import json
import pandas as pd
from tqdm import tqdm
import numpy as np

# Пути к файлам
REVIEW_PATH = "/content/drive/MyDrive/RecSys project T-Bank/Electronics.jsonl.gz"
META_PATH = "/content/drive/MyDrive/RecSys project T-Bank/meta_Electronics.jsonl.gz"

# Параметры фильтрации
MAX_REVIEWS = 250_000
MIN_USER_INTERACTIONS = 5
MIN_ITEM_INTERACTIONS = 5
MIN_RATING = 3.0

# Чтение отзывов
reviews_data = []
with gzip.open(REVIEW_PATH, "rt", encoding="utf-8") as f:
    for i, line in enumerate(tqdm(f, total=MAX_REVIEWS)):
        if i >= MAX_REVIEWS:
            break
        r = json.loads(line)
        timestamp = (
            r.get("sort_timestamp")
            or r.get("timestamp")
            or r.get("unixReviewTime")
        )
        if timestamp is None:
            continue
        reviews_data.append({
            "user_id": r["user_id"],
            "item_id": r["parent_asin"],
            "rating": r["rating"],
            "text": r.get("text", ""),
            "timestamp": pd.to_datetime(timestamp, unit='s', errors='coerce')
                         if isinstance(timestamp, (int, float))
                         else pd.to_datetime(timestamp, errors='coerce'),
            "verified": r.get("verified_purchase", False)
        })

interactions = pd.DataFrame(reviews_data)
print("Всего отзывов:", len(interactions))

# Фильтрация
interactions = interactions[interactions["rating"] >= MIN_RATING]

user_counts = interactions["user_id"].value_counts()
interactions = interactions[
    interactions["user_id"].isin(user_counts[user_counts >= MIN_USER_INTERACTIONS].index)
]

item_counts = interactions["item_id"].value_counts()
interactions = interactions[
    interactions["item_id"].isin(item_counts[item_counts >= MIN_ITEM_INTERACTIONS].index)
]

print("Отзывы после фильтрации:", len(interactions))

# Чтение метаданных
meta_dict = {}
with gzip.open(META_PATH, "rt", encoding="utf-8") as f:
    for line in tqdm(f):
        m = json.loads(line)
        meta_dict[m["parent_asin"]] = {
            "title": m.get("title"),
            "price": pd.to_numeric(m.get("price"), errors='coerce'),
            "avg_rating": m.get("average_rating"),
            "rating_number": m.get("rating_number"),
            "store": m.get("store"),
            "category": m.get("main_category")
        }

items = (
    interactions[["item_id"]]
    .drop_duplicates()
    .merge(
        pd.DataFrame.from_dict(meta_dict, orient="index"),
        left_on="item_id",
        right_index=True,
        how="left"
    )
)
items['price'].fillna(items['price'].median(), inplace=True)

# Расширенные фичи для взаимодействий
interactions['review_length'] = interactions['text'].apply(lambda x: len(x))
interactions['review_words'] = interactions['text'].apply(lambda x: len(x.split()))
interactions['is_high_rating'] = (interactions['rating'] >= 4).astype(int)
interactions['hour_of_day'] = interactions['timestamp'].dt.hour
interactions['day_of_week'] = interactions['timestamp'].dt.dayofweek

# Фичи пользователей
user_agg = interactions.groupby('user_id').agg(
    user_review_count=('item_id', 'count'),
    user_avg_rating=('rating', 'mean'),
    user_verified_ratio=('verified', 'mean'),
    user_review_length_avg=('review_length', 'mean'),
    user_first_review_ts=('timestamp', 'min'),
    user_last_review_ts=('timestamp', 'max')
).reset_index()

# Фичи офферов
item_agg = interactions.groupby('item_id').agg(
    offer_review_count=('user_id', 'count'),
    offer_avg_review_length=('review_length', 'mean'),
    offer_verified_ratio=('verified', 'mean')
).reset_index()

# Объединяем офферы с мета-данными
items = items.merge(item_agg, left_on='item_id', right_on='item_id', how='left')

# Переименование под банковский контекст
interactions.rename(columns={
    "user_id": "customer_id",
    "item_id": "offer_id",
    "rating": "offer_rating",
    "verified": "verified_transaction"
}, inplace=True)

items.rename(columns={
    "item_id": "offer_id",
    "avg_rating": "offer_avg_rating",
    "rating_number": "offer_rating_count",
    "category": "offer_category",
    "title": "offer_title"
}, inplace=True)

# Сортировка по времени
interactions.sort_values("timestamp", inplace=True)
interactions.reset_index(drop=True, inplace=True)
items.reset_index(drop=True, inplace=True)
user_agg.sort_values("user_id", inplace=True)
user_agg.reset_index(drop=True, inplace=True)

# Сохранение
interactions.to_parquet("/content/drive/MyDrive/RecSys project T-Bank/interactions_features.parquet")
items.to_parquet("/content/drive/MyDrive/RecSys project T-Bank/items_features.parquet")
user_agg.to_parquet("/content/drive/MyDrive/RecSys project T-Bank/users_features.parquet")

print("Interactions подготовлены:", len(interactions))
print("Items подготовлены:", len(items))
print("Users подготовлены:", len(user_agg))

100%|██████████| 250000/250000 [00:38<00:00, 6425.50it/s]


Всего отзывов: 250000
Отзывы после фильтрации: 55677


1610012it [02:14, 12002.95it/s]
/tmp/ipython-input-2870703770.py:84: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  items['price'].fillna(items['price'].median(), inplace=True)


Interactions подготовлены: 55677
Items подготовлены: 4739
Users подготовлены: 11074


Для обеспечения устойчивости пайплайна использовалась гибкая обработка временных меток, учитывающая различия в формате данных между категориями и версиями датасета:


```
timestamp = (
    r.get("sort_timestamp")
    or r.get("timestamp")
    or r.get("unixReviewTime")
)
```



### Очистка и фильтрация

Для повышения качества и стабильности обучения моделей выполняются следующие действия:

	•	удаляются пользователи с малым числом взаимодействий (например, меньше 5);
	•	удаляются товары с недостаточным количеством отзывов (например, меньше 5);
	•	оставляются только релевантные отзывы (рейтинг ≥ 3), что позволяет при необходимости преобразовать данные в формат implicit-feedback;
	•	приводятся числовые колонки (price) к корректному типу, некорректные значения заменяются на медиану;
	•	данные сортируются по времени, чтобы подготовить их для дальнейшего временного разбиения на train/validation/test.


In [ ]:
# Параметры фильтрации
MIN_USER_INTERACTIONS = 5
MIN_ITEM_INTERACTIONS = 5
MIN_RATING = 3.0

# Фильтрация отзывов
# Оставляем только релевантные отзывы (rating >= MIN_RATING)
interactions = interactions[interactions["offer_rating"] >= MIN_RATING]

# Удаляем пользователей с малым числом взаимодействий
user_counts = interactions["customer_id"].value_counts()
interactions = interactions[
    interactions["customer_id"].isin(user_counts[user_counts >= MIN_USER_INTERACTIONS].index)
]

# Удаляем товары с недостаточным количеством отзывов
item_counts = interactions["offer_id"].value_counts()
interactions = interactions[
    interactions["offer_id"].isin(item_counts[item_counts >= MIN_ITEM_INTERACTIONS].index)
]

# Обработка числовых колонок
# Приведение price к числовому типу, некорректные значения -> NaN
items['price'] = pd.to_numeric(items['price'], errors='coerce')
# Заполнение пропусков медианой
items['price'] = items['price'].fillna(items['price'].median())

# Сортировка по времени
interactions = interactions.sort_values("timestamp").reset_index(drop=True)
items = items.reset_index(drop=True)

# Сохранение чистых таблиц
interactions.to_parquet("/content/drive/MyDrive/RecSys project T-Bank/interactions_clean.parquet")
items.to_parquet("/content/drive/MyDrive/RecSys project T-Bank/items_clean.parquet")

print("Interactions после фильтрации:", len(interactions))
print("Items после фильтрации:", len(items))

Interactions после фильтрации: 32111
Items после фильтрации: 4739


### Проверка данных

На этом этапе выполняется анализ данных, чтобы понять их структуру и подготовить под задачу рекомендательной системы:

	•	просматриваем первые несколько строк таблиц interactions и items для визуального понимания;
	•	проверяем типы колонок и наличие пропусков;
	•	анализируем распределение рейтингов пользователей;
	•	считаем количество уникальных пользователей и товаров;
	•	смотрим статистику по ценам и другим числовым признакам товаров;
	•	на основе этого анализа будем корректировать данные, оставляя только ключевые признаки для моделей LightFM и LGBM.

In [ ]:
print(items.columns)
display(items.head())

Index(['item_id', 'title', 'price', 'avg_rating', 'rating_number', 'store',
       'category'],
      dtype='object')


,item_id,title,price,avg_rating,rating_number,store,category
0,B07CML419K,"Fancii Small Personal Desk USB Fan, Portable M...",15.99,4.0,11448,Fancii,Amazon Home
1,B07BHHB5RH,"Bluetooth Headphones, Soundcore Spirit Sports ...",24.99,4.1,2126,Anker,Cell Phones & Accessories
2,B09S6Y5BRG,Otium Bluetooth Earbuds Wireless Headphones Bl...,19.99,4.3,16471,Otium,All Electronics
3,B01LW71IBJ,Logitech Z313 Speaker System + Logitech Blueto...,24.99,4.4,14209,Logitech,All Electronics
4,B017T99JPG,Bose SoundLink Mini Bluetooth Speaker II (Carbon),170.00,4.5,6463,Bose,Home Audio & Theater


In [ ]:
# Просмотр первых строк
print("Примеры interactions:")
display(interactions.head())

print("\nПримеры items:")
display(items.head())

# Типы колонок и пропуски
print("Информация по interactions:")
interactions.info()

print("\nИнформация по items:")
items.info()

# Распределение рейтингов
print("\nРаспределение рейтингов пользователей:")
print(interactions['offer_rating'].value_counts())

# Количество уникальных пользователей и товаров
num_users = interactions['customer_id'].nunique()
num_items = interactions['offer_id'].nunique()
print(f"\nКоличество уникальных пользователей: {num_users}")
print(f"Количество уникальных товаров: {num_items}")

# Статистика по числовым колонкам
print("\nСтатистика по рейтингу и review_length:")
display(interactions[['offer_rating', 'review_length', 'review_words']].describe())

print("\nСтатистика по цене товаров:")
display(items['price'].describe())

print("\nСтатистика по рейтингу товаров:")
display(items['offer_avg_rating'].describe())
display(items['offer_rating_count'].describe())

# Выбор ключевых признаков для моделей
# то есть оставляем только признаки, которые будут использоваться в LightFM / LGBM
key_interaction_features = [
    'customer_id', 'offer_id', 'offer_rating', 'verified_transaction',
    'review_length', 'review_words', 'is_high_rating', 'hour_of_day', 'day_of_week'
]

key_item_features = [
    'offer_id', 'offer_title', 'price', 'offer_avg_rating', 'offer_rating_count',
    'offer_store', 'offer_category', 'offer_review_count', 'offer_avg_review_length', 'offer_verified_ratio'
]

print("\nВыбранные признаки для interactions:", key_interaction_features)
print("Выбранные признаки для items:", key_item_features)

Примеры interactions:


,customer_id,offer_id,offer_rating,text,timestamp,verified_transaction,review_length,review_words,is_high_rating,hour_of_day,day_of_week
0,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,B09S6Y5BRG,5.0,These are fantastic headphones and I love that...,NaT,True,1048,206,1,NaN,NaN
1,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,B01LW71IBJ,5.0,pretty good for the price.,NaT,True,26,5,1,NaN,NaN
2,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,B017T99JPG,5.0,yes.. so good. just buy it. my favorite featu...,NaT,True,94,17,1,NaN,NaN
3,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,B01HHURN3W,3.0,"Works well, but the corner of the plastic crac...",NaT,True,187,37,0,NaN,NaN
4,AEFKF6R2GUSK2AWPSWRR4ZO36JVQ,B07Q585X37,4.0,nice to have an accident plan just in case of ...,NaT,True,66,15,1,NaN,NaN



Примеры items:


,offer_id,offer_title,price,offer_avg_rating,offer_rating_count,store,offer_category,offer_review_count,offer_avg_review_length,offer_verified_ratio
0,B07CML419K,"Fancii Small Personal Desk USB Fan, Portable M...",15.990,4.0,11448,Fancii,Amazon Home,8,255.500000,1.000000
1,B09S6Y5BRG,Otium Bluetooth Earbuds Wireless Headphones Bl...,19.990,4.3,16471,Otium,All Electronics,6,328.166667,1.000000
2,B01LW71IBJ,Logitech Z313 Speaker System + Logitech Blueto...,23.845,4.4,14209,Logitech,All Electronics,18,357.833333,0.833333
3,B017T99JPG,Bose SoundLink Mini Bluetooth Speaker II (Carbon),170.000,4.5,6463,Bose,Home Audio & Theater,13,225.307692,0.769231
4,B01HHURN3W,Besdata Magnetic Smart Cover & Translucent Bac...,23.845,4.2,9669,Besdata,Computers,9,212.000000,1.000000


Информация по interactions:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32111 entries, 0 to 32110
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           32111 non-null  object        
 1   offer_id              32111 non-null  object        
 2   offer_rating          32111 non-null  float64       
 3   text                  32111 non-null  object        
 4   timestamp             0 non-null      datetime64[ns]
 5   verified_transaction  32111 non-null  bool          
 6   review_length         32111 non-null  int64         
 7   review_words          32111 non-null  int64         
 8   is_high_rating        32111 non-null  int64         
 9   hour_of_day           0 non-null      float64       
 10  day_of_week           0 non-null      float64       
dtypes: bool(1), datetime64[ns](1), float64(3), int64(3), object(3)
memory usage: 2.5+ MB

Информация по item

...

,price
count,4739.000000
mean,47.883044
std,86.332962
min,1.990000
25%,15.990000
50%,23.845000
75%,39.640000
max,1799.000000



Статистика по рейтингу товаров:


,offer_avg_rating
count,4739.000000
mean,4.448154
std,0.249182
min,2.700000
25%,4.300000
50%,4.500000
75%,4.600000
max,4.900000


,offer_rating_count
count,4.739000e+03
mean,1.542904e+04
std,3.852207e+04
min,7.000000e+00
25%,2.294500e+03
50%,5.758000e+03
75%,1.435300e+04
max,1.034896e+06



Выбранные признаки для interactions: ['customer_id', 'offer_id', 'offer_rating', 'verified_transaction', 'review_length', 'review_words', 'is_high_rating', 'hour_of_day', 'day_of_week']
Выбранные признаки для items: ['offer_id', 'offer_title', 'price', 'offer_avg_rating', 'offer_rating_count', 'offer_store', 'offer_category', 'offer_review_count', 'offer_avg_review_length', 'offer_verified_ratio']


### Разбиение данных и подготовка пользовательско-товарных матриц

На этом этапе выполняется:

	•	Разбиение взаимодействий пользователей на train / validation / test с учетом временной последовательности для каждого пользователя (70% / 15% / 15%).
	•	Создание разреженных user-item матриц для моделей LightFM (candidate generation), где строки — пользователи, колонки — товары, а значения — бинарные взаимодействия.
	•	Подготовка таблицы признаков товаров для LGBM (ranking), включая цену, средний рейтинг и количество отзывов, с заполнением пропущенных значений.
	•	Сохранение всех разбиений и матриц для дальнейшей тренировки рекомендательной системы.

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

# Пути к данным
DATA_DIR = "/content/drive/MyDrive/RecSys project T-Bank"
INTERACTIONS_FILE = "interactions_clean.parquet"
ITEMS_FILE = "items_features.parquet"

interactions_path = os.path.join(DATA_DIR, INTERACTIONS_FILE)
items_path = os.path.join(DATA_DIR, ITEMS_FILE)

# Загружаем данные
interactions = pd.read_parquet(interactions_path)
items = pd.read_parquet(items_path)

# Если нет timestamp, создаем фиктивный ранжированный индекс
if 'timestamp' not in interactions.columns or interactions['timestamp'].isnull().all():
    interactions['timestamp'] = np.arange(len(interactions))

# Разбиение interactions на train/validation/test по времени для каждого пользователя
interactions['rank_timestamp'] = interactions.groupby('customer_id')['timestamp'].rank(method='first', ascending=True)

train_idx = interactions['rank_timestamp'] <= interactions.groupby('customer_id')['rank_timestamp'].transform(lambda x: np.percentile(x, 70))
val_idx = (interactions['rank_timestamp'] > interactions.groupby('customer_id')['rank_timestamp'].transform(lambda x: np.percentile(x, 70))) & \
          (interactions['rank_timestamp'] <= interactions.groupby('customer_id')['rank_timestamp'].transform(lambda x: np.percentile(x, 85)))
test_idx = interactions['rank_timestamp'] > interactions.groupby('customer_id')['rank_timestamp'].transform(lambda x: np.percentile(x, 85))

train = interactions[train_idx].copy()
val = interactions[val_idx].copy()
test = interactions[test_idx].copy()

# Создание user-item маппинга
user_map = {uid: i for i, uid in enumerate(interactions['customer_id'].unique())}
item_map = {iid: i for i, iid in enumerate(interactions['offer_id'].unique())}

def build_sparse(df):
    rows = df['customer_id'].map(user_map).to_numpy()
    cols = df['offer_id'].map(item_map).to_numpy()
    data = df['is_high_rating'].to_numpy()  # бинарный interaction
    return coo_matrix((data, (rows, cols)), shape=(len(user_map), len(item_map)))

train_matrix = build_sparse(train)
val_matrix = build_sparse(val)
test_matrix = build_sparse(test)

# Подготовка признаков товаров для LGBM
item_features = items.copy()
numeric_cols = ['price', 'offer_avg_rating', 'offer_rating_count', 'offer_review_count', 'offer_avg_review_length', 'offer_verified_ratio']
for col in numeric_cols:
    if col in item_features.columns:
        item_features[col].fillna(item_features[col].median(), inplace=True)

# Сохраняем разбиение и матрицы
train.to_parquet(os.path.join(DATA_DIR, "train.parquet"))
val.to_parquet(os.path.join(DATA_DIR, "val.parquet"))
test.to_parquet(os.path.join(DATA_DIR, "test.parquet"))

np.save(os.path.join(DATA_DIR, "train_matrix.npy"), train_matrix.toarray())
np.save(os.path.join(DATA_DIR, "val_matrix.npy"), val_matrix.toarray())
np.save(os.path.join(DATA_DIR, "test_matrix.npy"), test_matrix.toarray())

item_features.to_parquet(os.path.join(DATA_DIR, "item_features_lgbm.parquet"))

print("Train interactions:", len(train))
print("Validation interactions:", len(val))
print("Test interactions:", len(test))
print("User-item матрицы и признаки товаров сохранены.")

/tmp/ipython-input-736689596.py:53: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  item_features[col].fillna(item_features[col].median(), inplace=True)


Train interactions: 21775
Validation interactions: 4298
Test interactions: 6038
User-item матрицы и признаки товаров сохранены.


## Промежуточный итог (работа с данными)

1.	Скачивание данных: выбрал Amazon Reviews 2023, категория Electronics, загрузил отзывы пользователей (review) и метаданные товаров (meta) в формате JSONL.
2.	Первичная обработка: распарсил файлы, выделил ключевые поля (user_id, item_id, rating, timestamp), объединил взаимодействия с метаданными товаров, удалил дубликаты и некорректные записи, привел рейтинги к бинарному формату для implicit-feedback.
3.	Очистка и фильтрация: удалил пользователей и товары с малым числом взаимодействий, обработал пропуски в признаках товаров, отсортировал данные по времени для корректного разбиения.
4.	Разбиение на train/validation/test: сделал временное разбиение (70/15/15) для каждого пользователя, чтобы модели могли обучаться на прошлых действиях и проверяться на будущих.
5.	Подготовка форматов для моделей:

	•	для LightFM создал разреженную user-item матрицу;
  
	•	для LGBM подготовил признаки товаров (цена, рейтинг, количество отзывов).

Все шаги выполнены для того, чтобы пайплайн был устойчивым и готовым к обучению двухуровневой модели — candidate generation на LightFM и re-ranking на LGBM.

## Работа с моделями

### Candidate Generation (Implicit ALS)

На этапе генерации кандидатов используется матричная факторизация для неявной обратной связи. Изначально рассматривалась модель **LightFM** с функцией потерь WARP, так как она хорошо подходит для implicit-feedback сценариев и напрямую оптимизирует ранжирование. Однако в среде Google Colab возникли ограничения на сборку и установку **LightFM**, связанные с зависимостями и компиляцией.

В связи с этим в проекте была использована библиотека implicit, а именно модель ALS (Alternating Least Squares), которая является стандартным и широко применяемым решением для задачи candidate generation в индустриальных рекомендательных системах.

Модель обучается на разреженной user–item матрице взаимодействий из train-выборки, где наличие взаимодействия интерпретируется как неявный положительный сигнал. ALS позволяет эффективно масштабироваться на большие данные и готовит основу для генерации потенциально релевантных товаров для каждого пользователя.

В этом коде были выполнены следующие шаги:

	•	преобразована train-матрица взаимодействий в разреженный формат CSR, требуемый библиотекой implicit;
	•	инициализирована модель ALS с 64 факторами, регуляризацией 0.01 и 15 итерациями;
	•	модель обучена только на train-данных, чтобы исключить утечку информации;
	•	в результате получены обученные user- и item-факторы, которые будут использоваться для генерации кандидатов на следующем этапе пайплайна.

In [ ]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for implicit: filename=implicit-0.7.2-cp312-cp312-linux_x86_64.whl size=933265 sha256=276a14fca313df0f0b50a44a8c925a03d78a2ea4d1d2570e7879bc42289d27d8
  Stored in directory: /root/.cache/pip/wheels/b2/00/4f/9ff8af07a0a53ac6007ea5d739da19cfe147a2df542b6899f8
Successfully built implicit


In [ ]:
from implicit.als import AlternatingLeastSquares
from scipy.sparse import coo_matrix

# Убедимся, что есть бинарный сигнал для implicit-feedback
if 'interaction' not in train.columns:
    train['interaction'] = (train['offer_rating'] >= 3).astype(int)

# Создаём разреженную матрицу user-item для ALS
user_map = {uid: i for i, uid in enumerate(train['customer_id'].unique())}
item_map = {iid: i for i, iid in enumerate(train['offer_id'].unique())}

rows = train['customer_id'].map(user_map).to_numpy()
cols = train['offer_id'].map(item_map).to_numpy()
data = train['interaction'].to_numpy()

train_matrix = coo_matrix((data, (rows, cols)), shape=(len(user_map), len(item_map)))

# Инициализация модели ALS
als_model = AlternatingLeastSquares(
    factors=64,
    regularization=0.01,
    iterations=15,
    calculate_training_loss=True
)

# Библиотека implicit ожидает, что матрица будет item-user
als_model.fit(train_matrix.T.tocsr())

# Сохраняем факторы для генерации кандидатов
user_factors = als_model.user_factors
item_factors = als_model.item_factors

print("ALS модель обучена.")
print("User factors shape:", user_factors.shape)
print("Item factors shape:", item_factors.shape)

/usr/local/lib/python3.12/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

ALS модель обучена.
User factors shape: (2897, 64)
Item factors shape: (3972, 64)


### Генерация топ-N кандидатов для пользователей

На этом этапе мы используем обученную ALS-модель из библиотеки implicit для генерации ограниченного списка потенциально интересных товаров для каждого пользователя. Для каждого пользователя предсказываются оценки релевантности всех товаров на основе обученных user- и item-факторов, после чего выбираются top‑K товаров с наибольшими оценками.

Формирование списка кандидатов выполняется на основе CSR-матрицы train-выборки. В отличие от изначального плана, на этом шаге мы не фильтровали уже просмотренные пользователем товары, чтобы сохранить полный набор потенциальных кандидатов для анализа и будущего обучения LGBM.

В ходе выполнения кода были выполнены следующие шаги:

	•	преобразована train‑матрица в формат CSR;
	•	инициализирована и обучена ALS-модель на train-данных;
	•	для каждого пользователя рассчитаны оценки релевантности для всех товаров;
	•	выбраны top‑K товаров с наивысшими оценками;
	•	результаты сохранены в словарь candidates, готовый для передачи на этап ранжирования LGBM.

Такой подход позволяет сократить пространство поиска с миллионов товаров до ограниченного числа релевантных кандидатов, обеспечивая LGBM более качественные данные для точного ранжирования.

In [ ]:
from implicit.als import AlternatingLeastSquares
from scipy.sparse import coo_matrix
import numpy as np
from tqdm import tqdm

TOP_K = 10  # количество кандидатов на пользователя

# Убедимся, что есть бинарный сигнал для implicit-feedback
if 'interaction' not in train.columns:
    train['interaction'] = (train['offer_rating'] >= 3).astype(int)

# Строим user/item маппинги
user_map = {uid: i for i, uid in enumerate(train['customer_id'].unique())}
item_map = {iid: i for i, iid in enumerate(train['offer_id'].unique())}

# Построение разреженной матрицы train (CSR)
rows = train['customer_id'].map(user_map).to_numpy()
cols = train['offer_id'].map(item_map).to_numpy()
data = train['interaction'].to_numpy()
train_csr = coo_matrix((data, (rows, cols)), shape=(len(user_map), len(item_map))).tocsr()

# Инициализация и обучение ALS
als_model = AlternatingLeastSquares(factors=64, regularization=0.01, iterations=15, random_state=42)
als_model.fit(train_csr)

print("ALS модель обучена.")
print("User factors shape:", als_model.user_factors.shape)
print("Item factors shape:", als_model.item_factors.shape)

# Подготовка списков пользователей и товаров в правильном порядке
user_ids = [uid for uid, idx in sorted(user_map.items(), key=lambda x: x[1])]
item_ids = [iid for iid, idx in sorted(item_map.items(), key=lambda x: x[1])]

# Генерация top-K кандидатов для каждого пользователя
candidates = {}
print("Генерация топ-K кандидатов для пользователей...")
for u_idx, uid in enumerate(tqdm(user_ids)):
    scores = als_model.user_factors[u_idx] @ als_model.item_factors.T
    top_indices = np.argpartition(-scores, TOP_K)[:TOP_K]  # выбираем TOP_K без полной сортировки
    top_items = [item_ids[i] for i in top_indices[np.argsort(-scores[top_indices])]]  # сортировка внутри топ-K
    candidates[uid] = top_items

print("\nГенерация кандидатов завершена.")

  0%|          | 0/15 [00:00<?, ?it/s]

ALS модель обучена.
User factors shape: (3972, 64)
Item factors shape: (2897, 64)
Генерация топ-K кандидатов для пользователей...


100%|██████████| 3972/3972 [00:00<00:00, 12587.17it/s]


Генерация кандидатов завершена.


### МИНИ ВЕРСИЯ

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# Параметры
topk = 10  # количество кандидатов на пользователя (пробовать увеличить постепенно)
mini_users = 10000  # количество пользователей для мини-версии
features = ["price", "avg_rating", "rating_number"]

# Подготовка датасета LGBM
items_df = items_cleaned.reset_index(drop=True)

# Ограничиваем кандидатов для mini-теста
selected_candidates = dict(list(candidates.items())[:mini_users])

user_list, item_list = [], []
for user_idx, item_indices in tqdm(selected_candidates.items(), desc="Формируем кандидатов"):
    user_list.extend([user_idx]*topk)
    item_list.extend(item_indices[:topk])

candidate_df = pd.DataFrame({
    "user_idx": user_list,
    "item_idx": item_list
})

user_ids_unique = interactions['user_id'].unique()
candidate_df["user_id"] = candidate_df["user_idx"].map(lambda x: user_ids_unique[x])
candidate_df["item_id"] = candidate_df["item_idx"].map(lambda x: items_df['item_id'].iloc[x])

candidate_df = candidate_df.merge(
    items_df[["item_id", "price", "avg_rating", "rating_number"]],
    on="item_id",
    how="left"
)

interactions_set = set(zip(interactions['user_id'], interactions['item_id']))
labels = [int((row.user_id, row.item_id) in interactions_set)
          for row in tqdm(candidate_df.itertuples(index=False), desc="Формируем метки")]

candidate_df = candidate_df.copy()
candidate_df.loc[:, "label"] = labels
candidate_df.loc[:, "price"] = candidate_df["price"].astype(float)
candidate_df.loc[:, "avg_rating"] = candidate_df["avg_rating"].astype(float)
candidate_df.loc[:, "rating_number"] = candidate_df["rating_number"].astype(int)

lgbm_dataset = candidate_df[["user_id", "item_id", "price", "avg_rating", "rating_number", "label"]]
print("Мини-LGBM dataset готов, размер:", lgbm_dataset.shape)

# Подготовка данных для LGBMRanker
X = lgbm_dataset[features].values
y = lgbm_dataset["label"].values
user_ids = lgbm_dataset["user_id"].values

unique_users = np.unique(user_ids)
train_users, val_users = train_test_split(unique_users, test_size=0.2, random_state=42)

train_mask = np.isin(user_ids, train_users)
val_mask = np.isin(user_ids, val_users)

X_train = X[train_mask]
y_train = y[train_mask]
X_val = X[val_mask]
y_val = y[val_mask]

groups_train = lgbm_dataset.loc[train_mask].groupby("user_id").size().to_numpy()
groups_val = lgbm_dataset.loc[val_mask].groupby("user_id").size().to_numpy()

# Инициализация и обучение LGBMRanker
lgbm_ranker = lgb.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="gbdt",
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

print("Начало обучения LGBMRanker на мини-датасете...")
lgbm_ranker.fit(
    X_train,
    y_train,
    group=groups_train,
    eval_set=[(X_val, y_val)],
    eval_group=[groups_val],
    eval_at=[3, 5],  # метрики для проверки качества ранжирования
    callbacks=[lgb.early_stopping(stopping_rounds=5), lgb.log_evaluation(1)]
)
print("Обучение завершено!")


In [ ]:
import numpy as np
from tqdm import tqdm

# мини-выборка для Colab
mini_lgbm_dataset = lgbm_dataset.sample(n=10000, random_state=42)

features = ["price", "avg_rating", "rating_number"]
X_pred = mini_lgbm_dataset[features]  # используем правильные feature names
user_ids = mini_lgbm_dataset["user_id"].values
item_ids = mini_lgbm_dataset["item_id"].values

# предсказания LGBMRanker
preds = lgbm_ranker.predict(X_pred)

# группируем предсказания по пользователям
user_pred_dict = {}
for u, i, p in zip(user_ids, item_ids, preds):
    if u not in user_pred_dict:
        user_pred_dict[u] = []
    user_pred_dict[u].append((i, p))

# выбираем top-K рекомендаций на пользователя
K = 10
user_topk = {
    u: [i for i, _ in sorted(items, key=lambda x: -x[1])[:K]]
    for u, items in user_pred_dict.items()
}

# ground truth (для мини-версии используем все interactions)
interactions_set_eval = set(zip(interactions['user_id'], interactions['item_id']))

# считаем Precision@K, Recall@K, NDCG@K
precision_list = []
recall_list = []
ndcg_list = []

for u, recs in tqdm(user_topk.items(), desc="Вычисляем метрики"):
    gt_count = sum(interactions['user_id'] == u)  # кол-во релевантных элементов
    hits = sum((u, i) in interactions_set_eval for i in recs)

    precision = hits / K
    recall = hits / max(1, gt_count)

    dcg = sum(1 / np.log2(idx + 2) for idx, i in enumerate(recs) if (u, i) in interactions_set_eval)
    idcg = sum(1 / np.log2(idx + 2) for idx in range(min(K, gt_count)))
    ndcg = dcg / idcg if idcg > 0 else 0

    precision_list.append(precision)
    recall_list.append(recall)
    ndcg_list.append(ndcg)

print(f"Mini Precision@{K}: {np.mean(precision_list):.4f}")
print(f"Mini Recall@{K}: {np.mean(recall_list):.4f}")
print(f"Mini NDCG@{K}: {np.mean(ndcg_list):.4f}")

### Подготовка датасета для LGBM

На этом этапе мы преобразовали кандидатов, сгенерированных моделью ALS, в датасет для обучения модели ранжирования. Для каждого пользователя и каждого кандидата сформированы строки с признаками товара (price, avg_rating, rating_number) и меткой взаимодействия (label), которая равна 1, если пользователь взаимодействовал с товаром в train/validation/test, иначе 0.

Выполнены следующие шаги:

	•	Созданы словари для сопоставления индексов ALS с реальными user_id и item_id.
	•	Сформированы обратные словари для быстрого поиска индексов по реальным ID.
	•	Использовано множество взаимодействий (user_id, item_id) для эффективного определения метки label.
	•	Пройден весь список кандидатов top-K для каждого пользователя и собраны признаки товаров в строки датасета.
	•	Полученные строки объединены в DataFrame, готовый для обучения модели LGBM.

В результате получился готовый LGBM-датасет с 1 575 800 строками и 6 колонками, который можно использовать для обучения модели ранжирования. Такой подход позволяет эффективно обучать supervised модель на ограниченном, но качественном множестве кандидатов, сохраняя информацию о взаимодействиях и ключевых характеристиках товаров.

In [ ]:
import pandas as pd
from tqdm import tqdm

# Создаем множество для быстрого поиска взаимодействий
interaction_set = set(zip(train['customer_id'], train['offer_id']))

# Список для хранения строк датасета LGBM
lgbm_rows = []

print("Формирование датасета для LGBM...")
for uid, top_items in tqdm(candidates.items()):
    for iid in top_items:
        label = 1 if (uid, iid) in interaction_set else 0
        item_feats = item_features[item_features['offer_id'] == iid].iloc[0]

        row = {
            'customer_id': uid,
            'offer_id': iid,
            'label': label,
            'price': item_feats['price'],
            'offer_avg_rating': item_feats['offer_avg_rating'],
            'offer_rating_count': item_feats['offer_rating_count'],
            'offer_review_count': item_feats['offer_review_count'],
            'offer_avg_review_length': item_feats['offer_avg_review_length'],
            'offer_verified_ratio': item_feats['offer_verified_ratio']
        }
        lgbm_rows.append(row)

# Создаем DataFrame
lgbm_df = pd.DataFrame(lgbm_rows)

print("Датасет для LGBM сформирован.")
print("Размерность:", lgbm_df.shape)
lgbm_df.head()

Формирование датасета для LGBM...


100%|██████████| 3972/3972 [00:40<00:00, 97.78it/s] 


Датасет для LGBM сформирован.
Размерность: (39720, 9)


,customer_id,offer_id,label,price,offer_avg_rating,offer_rating_count,offer_review_count,offer_avg_review_length,offer_verified_ratio
0,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,B09V1FT19S,0,28.99,4.8,337225,53,146.566038,1.000000
1,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,B00FB50SBU,0,18.30,4.7,82733,73,174.109589,0.986301
2,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,B0BGS23YKX,0,11.99,4.7,157069,79,123.911392,0.949367
3,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,B0BQV27MZ7,0,33.51,4.1,624,12,596.416667,0.250000
4,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,B0BGPJNJBH,0,20.93,4.3,63,17,833.176471,0.000000


In [ ]:
# Проверяем баланс классов
label_counts = lgbm_df['label'].value_counts()
total = len(lgbm_df)
print("Баланс классов:")
for label, count in label_counts.items():
    print(f"label = {label}: {count} ({count / total:.2%})")

Баланс классов:
label = 0: 32727 (82.39%)
label = 1: 6993 (17.61%)


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

items_df = items_cleaned.reset_index(drop=True)
topk = 10

user_list = []
item_list = []

for user_idx, item_indices in tqdm(candidates.items(), desc="Формируем кандидатов"):
    user_list.extend([user_idx]*topk)
    item_list.extend(item_indices[:topk])

candidate_df = pd.DataFrame({
    "user_idx": user_list,
    "item_idx": item_list
})

user_ids_unique = interactions['user_id'].unique()
candidate_df["user_id"] = candidate_df["user_idx"].map(lambda x: user_ids_unique[x])
candidate_df["item_id"] = candidate_df["item_idx"].map(lambda x: items_df['item_id'].iloc[x])

candidate_df = candidate_df.merge(
    items_df[["item_id", "price", "avg_rating", "rating_number"]],
    on="item_id",
    how="left"
)

interactions_set = set(zip(interactions['user_id'], interactions['item_id']))

labels = [int((row.user_id, row.item_id) in interactions_set)
          for row in tqdm(candidate_df.itertuples(index=False), desc="Формируем метки")]

candidate_df = candidate_df.copy()
candidate_df.loc[:, "label"] = labels
candidate_df.loc[:, "price"] = candidate_df["price"].astype(float)
candidate_df.loc[:, "avg_rating"] = candidate_df["avg_rating"].astype(float)
candidate_df.loc[:, "rating_number"] = candidate_df["rating_number"].astype(int)

lgbm_dataset = candidate_df[["user_id", "item_id", "price", "avg_rating", "rating_number", "label"]]

print("LGBM dataset готов, размер:", lgbm_dataset.shape)

### Обучение LGBMRanker

Что делаем: Используем LGBM с objective lambdarank для обучения на сгенерированных кандидатов с учетом группировки по пользователю.
Цель: Научиться правильно ранжировать товары для каждого пользователя, учитывая как прошлые взаимодействия, так и характеристики товара.

In [ ]:
# Уникальные пользователи в train и val
train_users = train['customer_id'].unique()
val_users = val['customer_id'].unique()

print(f"Количество уникальных пользователей в train: {len(train_users)}")
print(f"Количество уникальных пользователей в val: {len(val_users)}\n")

# Уникальные товары в train и val
train_items = train['offer_id'].unique()
val_items = val['offer_id'].unique()

print(f"Количество уникальных товаров в train: {len(train_items)}")
print(f"Количество уникальных товаров в val: {len(val_items)}\n")

# Проверка распределения числа взаимодействий на пользователя
train_group_sizes = train.groupby('customer_id').size()
val_group_sizes = val.groupby('customer_id').size()

print("Минимальное/максимальное число взаимодействий на пользователя:")
print(f"  train: {train_group_sizes.min()} - {train_group_sizes.max()}")
print(f"  val  : {val_group_sizes.min()} - {val_group_sizes.max()}")

Количество уникальных пользователей в train: 3972
Количество уникальных пользователей в val: 3138

Количество уникальных товаров в train: 2897
Количество уникальных товаров в val: 1844

Минимальное/максимальное число взаимодействий на пользователя:
  train: 1 - 100
  val  : 1 - 21


In [ ]:
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# Фичи для ранжирования
features = [
    "price",
    "offer_avg_rating",
    "offer_rating_count",
    "offer_review_count",
    "offer_avg_review_length",
    "offer_verified_ratio"
]

X = lgbm_df[features].values
y = lgbm_df["label"].values
user_ids = lgbm_df["customer_id"].values

# Разделяем пользователей на train и val
unique_users = np.unique(user_ids)
train_users, val_users = train_test_split(unique_users, test_size=0.2, random_state=42)

# Создаем маски для train и val
train_mask = np.isin(user_ids, train_users)
val_mask = np.isin(user_ids, val_users)

X_train, y_train = X[train_mask], y[train_mask]
X_val, y_val = X[val_mask], y[val_mask]

# Размер групп для LGBMRanker (число кандидатов на пользователя)
groups_train = lgbm_df.loc[train_mask].groupby("customer_id").size().to_numpy()
groups_val = lgbm_df.loc[val_mask].groupby("customer_id").size().to_numpy()

# Инициализация LGBMRanker
lgbm_ranker = lgb.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="gbdt",
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

print("Начало обучения LGBMRanker...")
lgbm_ranker.fit(
    X_train, y_train,
    group=groups_train,
    eval_set=[(X_val, y_val)],
    eval_group=[groups_val],
    eval_at=[3, 5],
    callbacks=[
        lgb.early_stopping(stopping_rounds=10),
        lgb.log_evaluation(period=5)
    ]
)
print("Обучение завершено!")

Начало обучения LGBMRanker...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002473 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 955
[LightGBM] [Info] Number of data points in the train set: 31770, number of used features: 6
Training until validation scores don't improve for 10 rounds
[5]	valid_0's ndcg@3: 0.688649	valid_0's ndcg@5: 0.73507
[10]	valid_0's ndcg@3: 0.697439	valid_0's ndcg@5: 0.739149
[15]	valid_0's ndcg@3: 0.699744	valid_0's ndcg@5: 0.740618
[20]	valid_0's ndcg@3: 0.699971	valid_0's ndcg@5: 0.744232
[25]	valid_0's ndcg@3: 0.700595	valid_0's ndcg@5: 0.744006
[30]	valid_0's ndcg@3: 0.699844	valid_0's ndcg@5: 0.74367
Early stopping, best iteration is:
[24]	valid_0's ndcg@3: 0.701222	valid_0's ndcg@5: 0.746838
Обучение завершено!


In [ ]:
len(lgbm_dataset['user_id'].unique())

### Предсказание и оценка качества модели

Что делаем:

	•	Прогоняем кандидатов через LGBM для получения предсказанных релевантностей;
	•	Вычисляем метрики Precision@K, Recall@K, NDCG@K.
Цель: Оценить качество рекомендательной системы, понять, насколько пользователи получают релевантные рекомендации.

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Признаки для модели
X_features = ["price", "offer_avg_rating", "offer_rating_count",
              "offer_review_count", "offer_avg_review_length", "offer_verified_ratio"]

# Кодируем пользователей и товары для группировки
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
lgbm_df['user_idx'] = user_encoder.fit_transform(lgbm_df['customer_id'])
lgbm_df['item_idx'] = item_encoder.fit_transform(lgbm_df['offer_id'])

# Предсказания модели
X = lgbm_df[X_features]  # передаем DataFrame с названиями колонок
y_pred = lgbm_ranker.predict(X)

# Собираем предсказания для каждого пользователя
TOP_K = 5
precision_list = []
recall_list = []
ndcg_list = []

for uid, group in lgbm_df.groupby('user_idx'):
    # сортировка по предсказанной релевантности
    top_items = group.iloc[np.argsort(-y_pred[group.index.values])][:TOP_K]

    # реальные релевантные (label=1)
    relevant = set(group[group['label'] == 1]['item_idx'])
    recommended = set(top_items['item_idx'])

    # Precision@K и Recall@K
    hits = len(recommended & relevant)
    precision_list.append(hits / TOP_K)
    recall_list.append(hits / max(len(relevant), 1))

    # NDCG@K
    dcg = sum([1 / np.log2(i+2) for i, idx in enumerate(top_items['item_idx']) if idx in relevant])
    idcg = sum([1 / np.log2(i+2) for i in range(min(len(relevant), TOP_K))])
    ndcg_list.append(dcg / max(idcg, 1))

# Средние метрики
print(f"Precision@{TOP_K}: {np.mean(precision_list):.4f}")
print(f"Recall@{TOP_K}   : {np.mean(recall_list):.4f}")
print(f"NDCG@{TOP_K}     : {np.mean(ndcg_list):.4f}")

Precision@5: 0.2591
Recall@5   : 0.6180
NDCG@5     : 0.5532


In [ ]:
# Рассчитываем Precision, Recall, NDCG для K=15
K = 15
precision_list, recall_list, ndcg_list = [], [], []

for uid, group in lgbm_df.groupby('user_idx'):
    top_items = group.iloc[np.argsort(-y_pred[group.index.values])][:K]
    relevant = set(group[group['label'] == 1]['item_idx'])
    recommended = set(top_items['item_idx'])

    hits = len(recommended & relevant)
    precision_list.append(hits / K)
    recall_list.append(hits / max(len(relevant), 1))

    dcg = sum([1 / np.log2(i+2) for i, idx in enumerate(top_items['item_idx']) if idx in relevant])
    idcg = sum([1 / np.log2(i+2) for i in range(min(len(relevant), K))])
    ndcg_list.append(dcg / max(idcg, 1))

prec15 = np.mean(precision_list)
rec15 = np.mean(recall_list)
ndcg15 = np.mean(ndcg_list)

# Средние метрики для K=15
print(f"Precision@15: {prec15:.4f}")
print(f"Recall@15   : {rec15:.4f}")
print(f"NDCG@15     : {ndcg15:.4f}")

Precision@15: 0.1174
Recall@15   : 0.7993
NDCG@15     : 0.6207


Анализ:

	1.	Precision@5 = 0.26 — примерно каждый 4-й рекомендованный товар в топ‑5 релевантен.
	2.	Recall@5 = 0.618 — модель покрывает около 62% релевантных товаров пользователя в топ‑5, что очень хорошо.
	3.	NDCG@5 = 0.553 — релевантные товары чаще оказываются в верхней части топ‑5.
	4.	Увеличение K до 15: Precision падает (логично, больше кандидатов — больше не релевантных), но Recall растет до 0.8, а NDCG тоже растет — модель лучше покрывает все релевантные товары и ранжирует их неплохо.

Вывод: система ALS + LGBMRanker демонстрирует реально полезные рекомендации. На следующем шаге можно улучшать:

	•	Feature engineering для товаров/пользователей,
	•	расширять candidate generation,
	•	оптимизировать гиперпараметры LGBM и ALS для повышения Precision@5

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Максимальное K
max_K = 15

# Списки для метрик
precision_list_all, recall_list_all, ndcg_list_all = [], [], []

# Предсказанные релевантности
y_pred_values = y_pred

# Генерация метрик для K=1..15
for K in range(1, max_K + 1):
    precision_list, recall_list, ndcg_list = [], [], []

    for uid, group in lgbm_df.groupby('user_idx'):
        top_items = group.iloc[np.argsort(-y_pred_values[group.index.values])][:K]
        relevant = set(group[group['label'] == 1]['item_idx'])
        recommended = set(top_items['item_idx'])

        hits = len(recommended & relevant)
        precision_list.append(hits / K)
        recall_list.append(hits / max(len(relevant), 1))

        dcg = sum([1 / np.log2(i + 2) for i, idx in enumerate(top_items['item_idx']) if idx in relevant])
        idcg = sum([1 / np.log2(i + 2) for i in range(min(len(relevant), K))])
        ndcg_list.append(dcg / max(idcg, 1))

    precision_list_all.append(np.mean(precision_list))
    recall_list_all.append(np.mean(recall_list))
    ndcg_list_all.append(np.mean(ndcg_list))

Ks = np.arange(1, max_K + 1)

# Построение интерактивного графика
fig = go.Figure()

fig.add_trace(go.Scatter(x=Ks, y=precision_list_all, mode='lines', name='Precision',
                         line=dict(color='#1f77b4', width=3)))
fig.add_trace(go.Scatter(x=Ks, y=recall_list_all, mode='lines', name='Recall',
                         line=dict(color='#ff7f0e', width=3, dash='dash')))
fig.add_trace(go.Scatter(x=Ks, y=ndcg_list_all, mode='lines', name='NDCG',
                         line=dict(color='#2ca02c', width=3, dash='dot')))

fig.update_layout(
    title='Метрики рекомендательной системы (Top-K до 15)',
    xaxis_title='Top-K',
    yaxis_title='Среднее значение метрики',
    xaxis=dict(tickmode='linear', dtick=1),
    yaxis=dict(range=[0, 1]),
    legend=dict(font=dict(size=12)),
    template='plotly_white'
)

fig.show()